# Time-frequency analysis tutorial


In this tutorial you'll learn how to explore the spectral content of the preprocessed epochs.

The analysis pipeline computed time-frequency representations (TFRs) from our Epochs. 
We’ll read these files with MNE and look at the power. 

In [8]:
import os

from mne.time_frequency import read_tfrs
from mne import grand_average
import numpy as np

from matplotlib import pyplot as plt

%matplotlib qt

## Read the data

In [9]:
# Change this path below to the path where the pipeline saved the .h5 files!
power_data_path = '/Users/weian/Downloads/Raw_data/preprocessed/power/power_asrt_longer/TRIPLET'

# List the files
power_H_file_names = sorted([f for f in os.listdir(power_data_path) if 'Day1_H' in f])
power_L_file_names = sorted([f for f in os.listdir(power_data_path) if 'Day1_L' in f])

# Read the data
power_H = [read_tfrs(os.path.join(power_data_path, f))[0] for f in power_H_file_names]
power_L = [read_tfrs(os.path.join(power_data_path, f))[0] for f in power_L_file_names]

Reading /Users/weian/Downloads/Raw_data/preprocessed/power/power_asrt_longer/TRIPLET/10_S_Day1_H_power-tfr.h5 ...
Reading /Users/weian/Downloads/Raw_data/preprocessed/power/power_asrt_longer/TRIPLET/11_S_Day1_H_power-tfr.h5 ...
Reading /Users/weian/Downloads/Raw_data/preprocessed/power/power_asrt_longer/TRIPLET/13_S_Day1_H_power-tfr.h5 ...
Reading /Users/weian/Downloads/Raw_data/preprocessed/power/power_asrt_longer/TRIPLET/14_S_Day1_H_power-tfr.h5 ...
Reading /Users/weian/Downloads/Raw_data/preprocessed/power/power_asrt_longer/TRIPLET/15_L_Day1_H_power-tfr.h5 ...
Reading /Users/weian/Downloads/Raw_data/preprocessed/power/power_asrt_longer/TRIPLET/16_L_Day1_H_power-tfr.h5 ...
Reading /Users/weian/Downloads/Raw_data/preprocessed/power/power_asrt_longer/TRIPLET/17_E_Day1_H_power-tfr.h5 ...
Reading /Users/weian/Downloads/Raw_data/preprocessed/power/power_asrt_longer/TRIPLET/1_S_Day1_H_power-tfr.h5 ...
Reading /Users/weian/Downloads/Raw_data/preprocessed/power/power_asrt_longer/TRIPLET/2_S_

## Compare power between the predictable and unpredictable stimuli

In [11]:
# Inspect grand average power
power_H_grand_average = grand_average(power_H)
power_L_grand_average = grand_average(power_L)

# Ignore subset of channels
ch_names =  [ch for ch in power_H_grand_average.info['ch_names'] if ch not in ['Fp1', 'Fp2', 
                                                                               'AF7', 'AF3',
                                                                               'AFz', 'AF8', 'AF4']]

power_H_grand_average = power_H_grand_average.pick_channels(ch_names).copy().crop(tmin=0.0)#.apply_baseline((-0.25, 0.0),)
power_L_grand_average = power_L_grand_average.pick_channels(ch_names).copy().crop(tmin=0.0)#.apply_baseline((-0.25, 0.0),)

Identifying common channels ...
Identifying common channels ...


## Time-frequency plots

In [12]:
# Matplotlib-based plots

# find where L power was lower than H
mask = np.mean(power_L_grand_average.data, axis=0) < np.mean(power_H_grand_average.data, axis=0)

times = 1e3 * power_H_grand_average.times  # change unit to ms

freqs = power_H_grand_average.freqs

fig, ax = plt.subplots(figsize=(16, 5), nrows=1, ncols=2, sharex='col', sharey='col')



ax[0].imshow(np.mean(power_L_grand_average.data * 1e6, axis=0),
           extent=[times[0], times[-1], freqs[0], freqs[-1]],
           aspect='auto', origin='lower', cmap='gray_r')

# Create new stats image with only significant clusters
# mask_plot = np.nan * np.ones_like(power_L_grand_average.data.mean(axis=0))
# mask_plot[mask] = power_L_grand_average.data.mean(axis=0)[mask]

# ax[0].imshow(mask_plot,
#            extent=[times[0], times[-1], freqs[0], freqs[-1]],
#            aspect='auto', origin='lower', cmap='viridis')


im = ax[1].imshow(np.mean(power_H_grand_average.data * 1e6, axis=0),
           extent=[times[0], times[-1], freqs[0], freqs[-1]],
           aspect='auto', origin='lower', cmap='gray_r')


ax[1].set_ylabel('')
ax[0].set_ylabel('Frequency [Hz]')
ax[0].set_xlabel('Time [ms]')
ax[0].set_title('L')
ax[1].set_title('H')
ax[1].set_xlabel('Time [ms]')

position=fig.add_axes([0.93,0.1,0.02,0.8])
fig.colorbar(im, cax=position)

fig.show()



In [13]:
# Built-in MNE plots

fig, ax = plt.subplots(figsize=(16, 5), nrows=1, ncols=2, sharex='col', sharey='col')

power_H_grand_average.copy().pick_channels(ch_names).plot(combine='mean',
                                                          axes=ax[0],
                                                          show=False,
                                                          colorbar=False)

# hide 1/f: use baseline (-0.25, 0.0) with mode "ratio" and use dB=True
power_L_grand_average.copy().pick_channels(ch_names).plot(combine='mean',
                                                          axes=ax[1],
                                                          show=False,
                                                          colorbar=False)

ax[1].set_ylabel('')
ax[0].set_title('L')
ax[1].set_title('H')


#position=fig.add_axes([0.93,0.1,0.02,0.8])
#fig.colorbar(im, cax=position)

fig.show()



No baseline correction applied
No baseline correction applied


In [17]:
#power_H_grand_average.copy().plot_topo(title='Average power')

In [15]:
# We will take the difference in power between the two conditions

difference_between_L_and_H = (power_L_grand_average.data * 1e6) - (power_H_grand_average.data * 1e6)

In [16]:
fig, ax = plt.subplots(figsize=(20, 4), nrows=1, ncols=3, sharex='col', sharey='col')

ax[0].imshow(np.mean(power_L_grand_average.data * 1e6, axis=0),
           extent=[times[0], times[-1], freqs[0], freqs[-1]],
           aspect='auto', origin='lower', cmap='viridis')

ax[1].imshow(np.mean(power_H_grand_average.data * 1e6, axis=0),
           extent=[times[0], times[-1], freqs[0], freqs[-1]],
           aspect='auto', origin='lower', cmap='viridis')


im = ax[2].imshow(np.mean(difference_between_L_and_H, axis=0),
           extent=[times[0], times[-1], freqs[0], freqs[-1]],
           aspect='auto', origin='lower', cmap='gray')



ax[1].set_ylabel('')
ax[0].set_ylabel('Frequency [Hz]')
ax[0].set_xlabel('Time [ms]')
ax[0].set_title('L')
ax[1].set_title('H')
ax[2].set_title('L - H')
ax[1].set_xlabel('Time [ms]')
ax[2].set_xlabel('Time [ms]')

position=fig.add_axes([0.93,0.1,0.015,0.8])
fig.colorbar(im, cax=position)

fig.show()